In [58]:
import yfinance as yf
import pandas as pd
import numpy as np

df = yf.download("EURGBP=X", start="2023-04-01", end="2023-07-09", interval="1d")

[*********************100%***********************]  1 of 1 completed


In [42]:
# Relative strength index manually

def rsi(df, periods = 14):
    close_delta = df['Close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

# rsi_values = rsi(df, 14)

In [43]:
# Commodity channel index

def calculate_cci(dataframe, period=20):
    typical_price = (dataframe['High'] + dataframe['Low'] + dataframe['Close']) / 3
    mean_deviation = typical_price.rolling(window=period).apply(lambda x: np.fabs(x - x.mean()).mean(), raw=True)
    cci = (typical_price - typical_price.rolling(window=period).mean()) / (0.015 * mean_deviation)
    return cci

#cci_values=calculate_cci(df, 20)

In [44]:
# Average true range

def calculate_atr(dataframe, period=14):
    high = dataframe['High']
    low = dataframe['Low']
    close = dataframe['Close']
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(span=period, adjust=False).mean()
    return atr

#atr_values=calculate_atr(df, 14)

In [59]:
# Using pandas technical indicators lib

import pandas_ta as ta

df["RSI"] = ta.rsi(df.Close, length=14)

df["CCI"] = ta.cci(df.High, df.Low, df.Close, length=14)

df["ATR"] = ta.atr(df.High, df.Low, df.Close, length=14)

df["AO"] = ta.ao(df.High, df.Low)

df["MOM"] = ta.mom(df.Close, length=14)

def cal_macd(df):
    macd = ta.macd(df.Close)
    df = df.join(macd)
    return df


